In [1]:
def extract_first_char(s: str):
    if s is None:
        raise ValueError("argument cannot be None")
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if len(s) == 0:
        raise ValueError("argument cannot be an empty string")
    return s[0]  

In [2]:
extract_first_char(100)

TypeError: argument must be a string

In [3]:
extract_first_char("")

ValueError: argument cannot be an empty string

In [4]:
from typing import Annotated

In [7]:
from pydantic import Field, validate_call

In [8]:
NonEmptyString = Annotated[str, Field(min_length=1)]

In [9]:
@validate_call
def extract_first_char(s: NonEmptyString):
    return s[0]  

In [10]:
extract_first_char(100)

ValidationError: 1 validation error for extract_first_char
0
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type

In [12]:
extract_first_char("")

ValidationError: 1 validation error for extract_first_char
0
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/string_too_short

In [11]:
extract_first_char("abc")

'a'

In [13]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse

def make_utc(dt: datetime) -> datetime:
    print("make_utc called...")
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    print("parse_datetime called...")
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

In [14]:
from pydantic import BeforeValidator, AfterValidator

DatetimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [15]:
@validate_call
def func(dt: DatetimeUTC):
    return dt.isoformat()

In [16]:
func("2020/1/1 3pm")

parse_datetime called...
make_utc called...


'2020-01-01T15:00:00+00:00'

In [17]:
def func(dt: datetime | str):
    if isinstance(dt, str):
        try:
            dt = parse(dt)
        except Exception as exc:
            raise ValueError(str(exc))
    return make_utc(dt)

In [18]:
func("2020/1/1")

make_utc called...


datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>)

In [19]:
func(100)

make_utc called...


ValueError: 'int' object has no attribute 'tzinfo'

In [20]:
func(datetime(2000, 1, 1))

make_utc called...


datetime.datetime(2000, 1, 1, 0, 0, tzinfo=<UTC>)